In [1]:
from scipy import signal
import lab11_helper
import numpy as np
import matplotlib.pyplot as plt
import timeit, time
%matplotlib inline

In [ ]:
# We define some useful constants:
pii = np.pi # Identifying pi 
j = np.complex(0,1) # identifying the complex number j

# Background

## What is the FFT?

In this lab, we'll implement the Fast Fourier Transform (FFT), an algorithm that computes the Discrete Fourier Transform (DFT) of a signal. Note that the DFT and FFT are *not* separate transforms: the FFT is just a specific algorithm for implementing the DFT, in the same way that quicksort is a specific algorithm for sorting an array. And, chances are, you've called a library function to sort an array before in Java or Python. But have you ever bothered to peruse the [source code for Java's Arrays.sort](http://developer.classpath.org/doc/java/util/Arrays-source.html) or for [Python's list sorting function](https://github.com/python/cpython/blob/master/Objects/listobject.c)? Probably not. And the beauty of well-designed abstractions is that you've never needed to - you can just call Arrays.sort (in Java) or `sort` (in Python) and everything's handled for you. Similarly, in practice, engineers don't care about the internals of `numpy.fft.fft` (numpy's function for computing the DFT), they just treat it as a black box for computing the DFT. However, in this lab, you'll get a chance to take a look inside the box, build your own, and compare it against the state of the art! This is well worth the effort given how widely used the FFT is in practical spectral analysis and as a building block for efficiently implementing dozens of other signal processing algorithms such as convolution, cross-correlation, image and audio compression, and more!

# Q1: Naive DFT

## Q1a: DFT Code

We'll begin by implementing the DFT the "naive" way by simply translating the DFT analysis equation into code. 

If we have a signal of duration $N$ (in a computer, a numpy array of length $N$), the DFT analysis equation is:

$$X[k] = \sum_{n = 0}^{N-1} x[n] e^{-j\frac{2\pi}{N}nk}$$

for $k, n \in \{0, 1, 2, ..., N - 1\}$. In general, $X[k]$ is a complex-valued array, even if $x[n]$ is purely real, which we must account for when writing our code by declaring our output type as `np.complex128` (a pair of floating-point "real" numbers, each 64 bits, for a total of 128 bits). If we don't, numpy will throw out the imaginary part when doing our computations, and return the incorrect result.

Fill in the function `dft` below to compute the DFT based on the analysis equation. In your array `X` that you output, `X[k], k = 0, 1, ..., N-1` should be the value of the `k`th DFT coefficient.

**Hint 1:** The function call `np.exp(z)` returns $e^z$, where $z$ is any number, real or complex. You've used this function before, but only on real numbers or arrays of real numbers. It works for complex inputs too!  
**Hint 2:** In Python, `1j` is used to represent the complex number $j$. For example, the Python expression `1j * 2 * np.pi` would evaluate to $2\pi j$. If you forget the 1 in front of the `j`, Python will think you're trying to reference a variable called `j` rather than the imaginary unit.

In [3]:
def dft(x):
    N = len(x)
    X = np.zeros(N, dtype=np.complex128) # output array
    ## TODO your code here ##
    
    for k in range(N):
        for n in range(N):
            X[k] += x[n]*np.exp(-1j*2*np.pi*n*k/N)
    ## TODO your code here ##
    return X

Let's do a couple quick sanity checks on our naive DFT function to make sure it returns the correct results. Run the cells below to check your work. If all tests do not print "True" for whether or not they passed, your `dft` function is buggy. 

Don't worry if it takes around half a minute (it shouldn't take more than 3-4 minutes to run all 5 of the tests) for the fourth and fifth tests to run - it's because we wrote a naive implementation of the DFT, which takes a long time for large inputs! By the end of this notebook, we'll have made big improvements, and hopefully these tests will make you appreciate the FFT a bit, as 1024 and 2048 are fairly commonly used data chunk sizes to take FFTs of in the real world.

This will take anywhere from 30 seconds to a few minutes to run depending on your laptop's processing power. This cell isn't graded on time, just correctness.

In [4]:
lab11_helper.run_fft_tests(dft)

Test 1 passed: True
Test 2 passed: True
Test 3 passed: True
Test 4 passed: True
Test 5 passed: True
Tests took 180.991 seconds


## Q1b: DFT Vectorized Code
Our implementation for the DFT function above was done in an iterative method, i.e. using Loops. This is not preferred due to the slowness and computation complexity of iterative algorithms. <br>

Another way to implement the DFT function is to use code vectorization. To understand code vectorization, let's compare the 2 different implementations of the following code, which calculates matrix multiplication.

In [5]:
# First Generate 2 matrix to be multiplied
x = np.arange(1, 10).reshape(3,3)  # Generate 3x3 matrix having values from 1 to 9
y = np.random.randint(0, 10, (3,1))  # Generate 3x1 matrix having random integers from 1 to 10

In [6]:
# Impliment matrix multiplication without using vecorization, i.e. using loops
def loop_dot(x, y):
    z = np.zeros((x.shape[0], y.shape[1]))
    for i in range(x.shape[0]):
        for j in range(y.shape[1]):
            z[i][j] = np.dot(x[i,:], y[:,j]) 
    return z

In [7]:
z = loop_dot(x, y)
print(z)

[[ 40.]
 [100.]
 [160.]]


Now we will use np.dot() function, which calculates the multiplication of 2 matrices using code vectorization.

In [8]:
# Vectorized Implimentation
z = np.dot(x, y)
print(z)

[[ 40]
 [100]
 [160]]


Now let's think how to impliment the vectorized DFT function.<br>
As a start we will begin with a vectorized code that calculates X[0] & X[1], then extend our implimentation to the whole DFT matrix. <br>

$$X[0] = \sum_{n = 0}^{N-1} x[n] e^{-j\frac{2\pi}{N}n (0)}$$
$$X[0] = \sum_{n = 0}^{N-1} x[n] $$
$$X[1] = \sum_{n = 0}^{N-1} x[n] e^{-j\frac{2\pi}{N}n (1)}$$
$$X[1] = x[0] e^{0} + x[1] e^{-j\frac{2\pi}{N}}+ x[2] e^{-j\frac{2\pi}{N} * 2} + .....$$

In [9]:
# Calulate X[0]
X_0 = np.sum(x)
print(X_0)

45


In [10]:
# Calculate X[1]
N = len(x) 
n = np.arange(0, N).reshape(1, N)
exp = np.exp(-j * 2 * pii / len(x) * n)  # Exponential matrix where n ranges from 0 to N-1
X_1 = np.sum(x * exp)
print("Vectorized Implimentation X[1] = ", X_1, "\nNon-Vectorized Implmenation - dft function - X[1] = ", dft(x)[1])

TypeError: only length-1 arrays can be converted to Python scalars

We shall extend our implementation to calculate the DFT matrix. As shown in the figure below, we need first to generate the DFT matrix. We can notice that we have 2 variables n & k that each has a range from 0 to N-1.

![DFT](./dft.JPG)

In [ ]:
def dft_vec(x):
    N = len(x)
    X = np.zeros(N, dtype=np.complex128) # output array
    ## TODO your code here ##
    n = np.arange(0, N).reshape(1, N)
    k = n
    print(np.dot(k.T, n))  # Matrix of n*k value to be included in the exponent of the DFT matrix
    dft = np.exp(2 * j * pii * np.dot(k.T, n))   # DFT matrix shown in the figure above
    X = np.dot(dft, x)
    ## TODO your code here ##
    return X

In [ ]:
X = dft_vec(x)
print(X)

# Q2: DCT (Discrete Cosine Transform)

In this section, you are required to calculate the DCT of a matrix, using both vectorized and non-vectorized methods.<br>

If we have a signal of duration $N$ (in a computer, a numpy array of length $N$), the DCT analysis equation is:

$$X[k] = \sqrt{2/N}  Ck \sum_{n = 0}^{N-1}  x[n] cos((2j+1)\frac{pi}{2N}k)$$
$$Ck = \frac{1}{\sqrt{2}}\ if\ k = 0$$
$$\ = 1\ otherwise$$


## Q2a: DFT Code

In [ ]:
# DCT without Vectorization
def dct(x):
    N = len(x)
    X = np.zeros(N, dtype=np.complex128) # output array
    ## TODO your code here ##
    CK=1/sqrt(2)
    for k in range(N):
        for n in range(N):
            X[k] += sqrt(2/N)*CK*np.cos((2j+1)*(pii/2*N)*k)
    ## TODO your code here ##
    return X

In [ ]:
X=dct(x)
print(X)

## Q2b: DFT Vectorized Code

In [ ]:
# DCT using Vectorization
def dct_vec(x):
    N = len(x)
    X = np.zeros(N, dtype=np.complex128) # output array
    ## TODO your code here ##
    n = np.arange(0, N).reshape(1, N)
    k = n
    print(np.dot(k.T, n))  # Matrix of n*k value to be included in the exponent of the DFT matrix
    dft = np.cos((2j+1)*(pii/2*N)*k)  # DFT matrix shown in the figure above
    X = np.dot(dft, x)
    ## TODO your code here ##
    return X

In [ ]:
import math
from math import sqrt


In [ ]:
X = dct_vec(x)
print(X)